# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data manipulation library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secretofthedev import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
# Legacy API
# api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

[{'avgTotalVolume': 63950500, 'calculationPrice': 'close', 'change': 0.76, 'changePercent': 0.0045, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': None, 'highTime': None, 'iexAskPrice': 169.11, 'iexAskSize': 238, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 169.06, 'iexCloseTime': 1712577936925, 'iexLastUpdated': 1712577936925, 'iexMarketPercent': 0.0010838498010152199, 'iexOpen': 169.05, 'iexOpenTime': 1712577936925, 'iexRealtimePrice': 169.05, 'iexRealtimeSize': 35, 'iexVolume': 259, 'lastTradeTime': 1712347199767, 'latestPrice': 169.58, 'latestSource': 'Close', 'latestTime': 'April 5, 2024', 'latestUpdate': 1712347200333, 'latestVolume': None, 'low': None, 'lowSource': 'Close', 'lowTime': 1712347200333, 'marketCap': 2618634179980, 'oddLotDelay

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data[0]['latestPrice']
market_cap = data[0]['marketCap']
print(price, market_cap/1000000000000)

169.58 2.61863417998


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe
print(type(final_dataframe))
final_dataframe

<class 'pandas.core.frame.DataFrame'>


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
new_row = [symbol, price, market_cap, 'N/A']
# convert list of rows into df
new_row_df = pd.DataFrame([new_row], columns=my_columns)
print(new_row_df)

final_dataframe = pd.concat([final_dataframe, new_row_df], ignore_index=True)
final_dataframe

  Ticker  Stock Price  Market Capitalization Number of Shares to Buy
0   AAPL       169.58          2618634179980                     N/A


/var/folders/_5/hcq3myc97jn9bf7mbp59_jn00000gn/T/ipykernel_39191/3212206648.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, new_row_df], ignore_index=True)


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,169.58,2618634179980,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    new_row = [stock, data[0]['latestPrice'], data[0]['marketCap'], 'N/A']
    new_row_df = pd.DataFrame([new_row], columns = my_columns)
    final_dataframe = pd.concat([final_dataframe, new_row_df], ignore_index = True)

/var/folders/_5/hcq3myc97jn9bf7mbp59_jn00000gn/T/ipykernel_39191/3576039709.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, new_row_df], ignore_index = True)


In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,144.12,42235127530,N/A
1,AAL,13.76,9009453788,N/A
2,AAP,77.34,4610196487,N/A
3,AAPL,169.58,2618634179980,N/A
4,ABBV,170.00,301009987110,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

symbol_string = ','.join(stocks['Ticker'].tolist())
batch_api_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
print(batch_api_url)

https://api.iex.cloud/v1/data/core/QUOTE/A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC

Fetch Data via Get Request - Batch API Request & Populate DF

In [11]:
data = requests.get(batch_api_url).json()

for stock in data:
    # print(stock['symbol'])
    # print(stock['latestPrice'])
    # print(stock['marketCap'])
    final_dataframe = pd.concat([final_dataframe, pd.DataFrame([[stock['symbol'], stock['latestPrice'], stock['marketCap'], 'N/A']], columns = my_columns)], ignore_index = True)
    
final_dataframe

# for i, symbol in enumerate(symbol_string.split(',')):
#     print(symbol)
#     print(data[i]['symbol'])
#     print(data[i]['latestPrice'])
#     print(data[i]['marketCap'])
    # final_dataframe = pd.concat([final_dataframe, pd.DataFrame([[symbol, data[symbol]['latestPrice'], data[symbol]['marketCap'], 'N/A']], columns = my_columns)], ignore_index = True)

# for symbol in symbol_string.split(','):
#     print(symbol)
#     final_dataframe = pd.concat([final_dataframe, pd.DataFrame([[symbol, data[symbol]['latestPrice'], data[symbol]['marketCap'], 'N/A']], columns = my_columns)], ignore_index = True)

/var/folders/_5/hcq3myc97jn9bf7mbp59_jn00000gn/T/ipykernel_39191/3837765038.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, pd.DataFrame([[stock['symbol'], stock['latestPrice'], stock['marketCap'], 'N/A']], columns = my_columns)], ignore_index = True)


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,144.12,42235127530,N/A
1,AAL,13.76,9009453788,N/A
2,AAP,77.34,4610196487,N/A
3,AAPL,169.58,2618634179980,N/A
4,ABBV,170.00,301009987110,N/A
...,...,...,...,...
500,YUM,136.10,38289867708,N/A
501,ZBH,127.33,26161406556,N/A
502,ZBRA,290.38,14924815052,N/A
503,ZION,42.08,6210966762,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [12]:
while True:
    try:
        portfolio_size = input('Enter the value of your portfolio:')
        val = float(portfolio_size)
        print(f'Portfolio value saved: {val}')
        break
    except ValueError:
        print('That is not a number. \nPlease try again.')
    

Portfolio value saved: 1000000.0


In [13]:
position_size = val/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])

print(len(final_dataframe.index))
print(position_size)

final_dataframe

505
1980.1980198019803


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,144.12,42235127530,13
1,AAL,13.76,9009453788,143
2,AAP,77.34,4610196487,25
3,AAPL,169.58,2618634179980,11
4,ABBV,170.00,301009987110,11
...,...,...,...,...
500,YUM,136.10,38289867708,14
501,ZBH,127.33,26161406556,15
502,ZBRA,290.38,14924815052,6
503,ZION,42.08,6210966762,47


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [14]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

/var/folders/_5/hcq3myc97jn9bf7mbp59_jn00000gn/T/ipykernel_39191/2598291447.py:2: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  final_dataframe.to_excel(writer, 'Recommended Trades', index = False)


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [16]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [17]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [18]:
writer.close()